In [39]:
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.metrics import plot_confusion_matrix
from tensorflow.keras.models import Sequential
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
import scipy
import scipy.integrate 
from IPython.display import Image
from keras import backend as K
from tensorflow import keras
import tensorflow as tf
import numpy as np
import itertools
import glob
import os

In [40]:
class Prediction_Histology:

    def __init__(self, image_gen, shape, model_call, rm):

        self.image_gen = image_gen
        self.shape = shape
        self.model_call = model_call
        self.rm = rm
        

    def preprocess(self):  

        os.chdir('/home/lebo/PyHIST')

        train_path = '/home/lebo/PyHIST/training'
        test_path = '/home/lebo/PyHIST/testing'
        validation_path = '/home/lebo/PyHIST/validation'

                
        train_path = ImageDataGenerator(preprocessing_function=self.image_gen) \
        .flow_from_directory(directory=train_path, target_size=self.shape,
        classes=['stg1', 'stg23', 'stg4'], batch_size=15)
        
        validation_batch = ImageDataGenerator(preprocessing_function=self.image_gen) \
        .flow_from_directory(directory=validation_path, target_size=self.shape,
        classes=['stg1', 'stg23', 'stg4'], batch_size=15)
        
        test_batch = ImageDataGenerator(preprocessing_function=self.image_gen) \
        .flow_from_directory(directory=test_path, target_size=self.shape,
        classes=['stg1', 'stg23', 'stg4'], batch_size=15)

    
        imgs, labels = next(train_batch)
        x = self.model_call.layers[self.rm].output

        config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=12,
                                inter_op_parallelism_threads=6,
                                allow_soft_placement=True,
                                device_count = {'CPU': 12})

        session = tf.compat.v1.Session(config=config)

        os.environ["6"] = "6"
        os.environ["10"] = "30"
        os.environ["KMP_SETTINGS"] = "1"
        os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"

        output = Dense(units=3, activation='softmax')(x)
        model = Model(inputs=self.model_call.input, outputs=output)

        model.compile(optimizer=Adam(learning_rate=0.0001),
                            loss='categorical_crossentropy', metrics=['accuracy'])

        history = model.fit(x=train_batch, validation_data=validate_batch, epochs=2, verbose=1)
        predictions = model.predict(x=validate_batch, verbose=0)


In [41]:
mobile_preprocess = tf.keras.applications.mobilenet.preprocess_input

In [42]:
mobileNet = tf.keras.applications.mobilenet.MobileNet()

In [43]:
hist_mobile = Prediction_Histology(mobile_preprocess, (224,224), mobileNet, -2)

In [ ]:
hist_mobile.preprocess()